In [ ]:
try:
    import RPi.GPIO as GPIO
except:
    import Mock.GPIO as GPIO

import time as thyme
from datetime import time, timedelta
from whendo.sdk.client import Client
import whendo.core.util as util
from whendo.core.util import DateTime, DateTime2, Now, KeyTagMode
import whendo.core.actions.file_action as file_x
import whendo.core.actions.list_action as list_x
import whendo.core.schedulers.timed_scheduler as timed_x
import whendo_gpio.scheduler as sched_x
import whendo.core.actions.dispatch_action as disp_x
from whendo.core.scheduler import Immediately
from whendo.core.server import Server
import whendo.core.programs.simple_program as prog_x
import whendo_gpio.action as gpio_x
import whendo.core.actions.sys_action as sys_x

"""
define the servers
"""
local_host = "192.168.0.26"
pi3 = Server(host="192.168.0.46", port=8000, tags={"server_name":["pi3"], "roles":["pivot"]})
pi4 = Server(host="192.168.0.45", port=8000, tags={"server_name":["pi4"], "roles":["pivot"]})
local = Server(host=local_host, port=8000, tags={"server_name":["local"], "roles":["hub"]})
all_servers = [pi3, pi4, local]
pivot_servers = [server for server in all_servers if server.has_tag("pivot")]
hub_servers = [server for server in all_servers if server.has_tag("hub")]

"""
define the actions
"""
gpio_clear = gpio_x.CleanupPins()
file_heartbeat1 = file_x.FileAppend(file="gpio_beat1.txt")
file_heartbeat2 = file_x.FileAppend(file="gpio_beat2.txt")
file_heartbeat3 = file_x.FileAppend(file="gpio_beat3.txt")
system_info = sys_x.SysInfo()
mini_info = sys_x.MiniInfo()
success = list_x.Success()
pause1 = sys_x.Pause()
pause2 = sys_x.Pause(seconds=2)
file_append = file_x.FileAppend(mode="d", file="boomerang.txt")
execute_action = disp_x.Exec(server_name="local", action_name="file_append")
raise_if_equal_to_0 = list_x.RaiseIfEqual(value=0)
raise_if_equal_to_1 = list_x.RaiseIfEqual(value=1)
boomerang = list_x.All(actions=[system_info, execute_action])

start_stop = DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=25))
start_stop_vals = list_x.Vals(vals={"start_stop": start_stop})

schedule_pivot = disp_x.ScheduleProgram(program_name="pivot_program1")
schedule_key_tags = disp_x.ExecKeyTags(key_tags={"roles":["pivot"]}, action_name="schedule_pivot")
schedule_key_tags2 = disp_x.ExecSuppliedKeyTags(key_tags={"roles":["pivot"]}, action = disp_x.ScheduleProgram(program_name="pivot_program1"))
schedule_key_tags3 = disp_x.ExecSuppliedKeyTags(key_tags={"roles":["pivot"]}, action = disp_x.ScheduleProgram(program_name="pivot_program1", start_stop=start_stop))
# schedule_key_tags3 = list_x.All(actions=[list_x.Vals(vals={"start_stop":start_stop}), schedule_key_tags2])

unschedule_pivot = disp_x.UnscheduleProgram(program_name="pivot_program1")
# unschedule_key_tags = disp_x.ExecKeyTags(key_tags={"roles":["pivot"]}, action_name="unschedule_pivot")
unschedule_key_tags = disp_x.ExecSuppliedKeyTags(key_tags={"roles":["pivot"]}, action = disp_x.UnscheduleProgram(program_name="pivot_program1"))

start_pivots = list_x.All(include_processing_info=True, actions=[unschedule_key_tags, start_stop_vals, schedule_key_tags])


green_pin = 27
green_on = gpio_x.SetPin(pin=green_pin, on=True)
green_off = gpio_x.SetPin(pin=green_pin, on=False)
green_toggle = gpio_x.TogglePin(pin=green_pin)
green_state = gpio_x.PinState(pin=green_pin)
green_change_append = file_x.FileAppend(mode="d", file="green_changes.txt")
green_pin_changes = list_x.All(actions=[mini_info, green_state, green_change_append])
green_boomerang = list_x.All(include_processing_info=True, actions=[mini_info, green_state, execute_action])
green_boomerang1 = list_x.All(include_processing_info=True, actions=[execute_action])
green_boomerang2 = list_x.All(include_processing_info=True, actions=[green_state, execute_action])
green_boomerang2a = list_x.All(include_processing_info=True, actions=[mini_info, execute_action])

red_pin = 25
red_on = gpio_x.SetPin(pin=red_pin, on=True)
red_off = gpio_x.SetPin(pin=red_pin, on=False)
red_toggle = gpio_x.TogglePin(pin=red_pin)
red_state = gpio_x.PinState(pin=red_pin)
red_change_append = file_x.FileAppend(mode="d", file="red_changes.txt")
red_pin_changes = list_x.All(actions=[mini_info, red_state, red_change_append])
red_boomerang = list_x.All(actions=[mini_info, red_state, execute_action])

toggle_toggle = list_x.All(actions=[green_toggle, red_toggle])
on_on = list_x.All(actions=[green_on, red_on])
off_off = list_x.All(actions=[green_off, red_off])
start_pivot1 = list_x.All(actions=[red_on, green_on, pause1, green_off])
start_pivot2 = list_x.All(actions=[green_on, red_on, pause1, red_off])

"""
define the schedulers
"""
often = timed_x.Timely(interval=1)
morning, evening = time(6,0,0), time(18,0,0)
daily_often = timed_x.Timely(start=morning, stop=evening, interval=1)
nightly_often = timed_x.Timely(start=evening, stop=morning, interval=1)
randomly_often = timed_x.Randomly(time_unit=util.TimeUnit.second, low=2, high=5)
timely = timed_x.Timely(interval=1)
timely_at_00_sec = timed_x.Timely(interval=1, second=00)
timely_at_30_sec = timed_x.Timely(interval=1, second=30)
timely_at_10_sec = timed_x.Timely(interval=1, second=10)
timely_every_5 = timed_x.Timely(interval=5)
timely_every_10 = timed_x.Timely(interval=10)
timely_every_30 = timed_x.Timely(interval=30)
immediately = Immediately()
green_rising_falling = sched_x.PinScheduler(pin=green_pin, event=GPIO.BOTH)
red_rising_falling = sched_x.PinScheduler(pin=red_pin, event=GPIO.BOTH)

"""
define the programs
"""
pivot_program1 = prog_x.PBEProgram().prologue("start_pivot1").epilogue("gpio_clear").body_element("timely_every_10", "red_on")
pivot_program2 = prog_x.PBEProgram().prologue("start_pivot2").epilogue("gpio_clear").body_element("timely_every_10", "green_on")
pivot_program3 = prog_x.PBEProgram().prologue("red_on").epilogue("gpio_clear").body_element("timely", "green_toggle")
pivot_program4 = prog_x.PBEProgram().prologue("green_on").epilogue("gpio_clear").body_element("timely", "red_toggle")
file_heartbeat_program = prog_x.PBEProgram().prologue("file_heartbeat1").epilogue("file_heartbeat3").body_element("timely", "file_heartbeat2")
toggle_program = prog_x.PBEProgram().body_element("timely_every_5", "toggle_toggle").epilogue("gpio_clear")

green_boomerang_program = prog_x.PBEProgram().body_element("timely_every_5", "green_boomerang")
green_callback_program = prog_x.PBEProgram().body_element("green_rising_falling", "green_boomerang")
green_toggle_program = prog_x.PBEProgram().body_element("timely_every_5", "green_toggle")
green_monitor_program = prog_x.PBEProgram().body_element("timely_every_5", "green_pin_changes")

red_boomerang_program = prog_x.PBEProgram().body_element("timely_every_5", "red_boomerang")
red_callback_program = prog_x.PBEProgram().body_element("red_rising_falling", "red_boomerang")
red_toggle_program = prog_x.PBEProgram().body_element("timely_every_5", "red_toggle")
red_monitor_program = prog_x.PBEProgram().body_element("timely_every_5", "red_pin_changes")



def client(s:Server):
    return Client(host=s.host, port=s.port)


def add_actions(s:Server):
    """
    add actions to the 'client' dispatcher
    """
    [ client(s).add_action(*action) for action in [
        ('gpio_clear', gpio_clear),
        ('toggle_toggle', toggle_toggle),
        ('on_on', on_on),
        ('off_off', off_off),
        ('file_heartbeat1', file_heartbeat1),
        ('file_heartbeat2', file_heartbeat2),
        ('file_heartbeat3', file_heartbeat3),
        ('sys_info', system_info),
        ('success', success),
        ('pause1', pause1),
        ('pause2', pause2),
        ('start_pivot1', start_pivot1),
        ('start_pivot2', start_pivot2),
        ('execute_action', execute_action),
        ('boomerang', boomerang),
        ('file_append', file_append),
        ('raise_if_equal_to_0', raise_if_equal_to_0),
        ('raise_if_equal_to_1', raise_if_equal_to_1),
        ('schedule_pivot', schedule_pivot),


        ('green_on', green_on),
        ('green_off', green_off),
        ('green_toggle', green_toggle),
        ('green_state', green_state),
        ('green_pin_changes', green_pin_changes),
        ('green_boomerang', green_boomerang),
        ('green_boomerang1', green_boomerang1),
        ('green_boomerang2', green_boomerang2),
        ('green_boomerang2a', green_boomerang2a),

        ('red_on', red_on),
        ('red_off', red_off),
        ('red_toggle', red_toggle),
        ('red_state', red_state),
        ('red_pin_changes', red_pin_changes),
        ('red_boomerang', red_boomerang),

        ('start_stop_vals', start_stop_vals),
        ('start_pivots', start_pivots),
        ('schedule_key_tags', schedule_key_tags),
        ('schedule_key_tags2', schedule_key_tags2),
        ('schedule_key_tags3', schedule_key_tags3)
        ]
    ]

def add_schedulers(s:Server):
    """
    add schedulers to the 'client' dispatcher
    """
    [ client(s).add_scheduler(*scheduler) for scheduler in [
        ('often', often),
        ('daily_often', daily_often),
        ('nightly_often', nightly_often),
        ('randomly_often', randomly_often),
        ('timely', timely),
        ('timely_at_00_sec', timely_at_00_sec),
        ('timely_at_10_sec', timely_at_10_sec),
        ('timely_at_30_sec', timely_at_30_sec),
        ('timely_every_5', timely_every_5),
        ('timely_every_10', timely_every_10),
        ('timely_every_30', timely_every_30),
        ('heartbeat', timely_every_30),
        ('immediately', immediately),
        ('green_rising_falling', green_rising_falling),
        ('red_rising_falling', red_rising_falling)
        ]
    ]

def add_programs(s:Server):
    """
    add programs to the 'client' dispatcher
    """
    [ client(s).add_program(*program) for program in [
        ('pivot_program1', pivot_program1),
        ('pivot_program2', pivot_program2),
        ('pivot_program3', pivot_program3),
        ('pivot_program4', pivot_program4),
        ('file_heartbeat_program', file_heartbeat_program),
        ('toggle_program', toggle_program),

        ('green_boomerang_program', green_boomerang_program),
        ('green_callback_program', green_callback_program),
        ('green_toggle_program', green_toggle_program),
        ('green_monitor_program', green_monitor_program),

        ('red_boomerang_program', red_boomerang_program),
        ('red_callback_program', red_callback_program),
        ('red_toggle_program', red_toggle_program),
        ('red_monitor_program', red_monitor_program)
        ]
    ]

def add_servers(s:Server):
    """
    add all servers to the 'client' dispatcher
    """
    for server in all_servers:
        client(s).add_server(server.get_tags_by_key("server_name")[0], server)

def replace_inventory(s:Server):
    client(s).clear_dispatcher()
    add_actions(s)
    # clear_gpio(s)
    add_schedulers(s)
    add_programs(s)
    add_servers(s)


def unschedule_schedulers(s):
    """
    unschedule the schedulers
    """
    client(s).unschedule_all_schedulers()
    
def all_on(s):
    client(s).execute_action('green_on')
    client(s).execute_action('red_on')

def all_off(s):
    client(s).execute_action('green_off')
    client(s).execute_action('red_off')

def sys_info(s):
    util.PP.pprint(client(s).execute_action('sys_info'))

def clear_gpio(s):
    client(s).execute_action("gpio_clear")

"""
convenience functions applied to a list of servers
"""
def schedule(svrs):
    # gpio_clr(svrs)
    for s in svrs:
        schedule_actions(s)
def unschedule():
    for s in pivot_servers:
        unschedule_schedulers(s)
def run(svrs):
    for s in svrs:
        client(s).run_jobs()
def stop(svrs):
    for s in svrs:
        client(s).stop_jobs()
def gpio_clr(svrs):
    for s in svrs:
        clear_gpio(s)
def on(svrs):
    for s in svrs:
        all_on(s)
def off(svrs):
    for s in svrs:
        all_off(s)

def inventory(svrs):
    for s in svrs:
        replace_inventory(s)
def pprint(svrs):
    for s in svrs:
        client(s).load_dispatcher().pprint()
def describe(svrs):
    for s in svrs:
        util.PP.pprint(client(s).describe_all())
def info(svrs):
    for s in svrs:
        sys_info(s)

def schedule_actions(s:Server):
    """
    schedule the actions of interest
    """
    c = client(s)
    def toggle(start_stop:DateTime2):
        c.schedule_program("green_toggle_program", start_stop=start_stop)
        c.schedule_program("red_toggle_program", start_stop=start_stop)
    def local_monitor(start_stop:DateTime2):
        c.schedule_program("green_monitor_program", start_stop=start_stop)
        c.schedule_program("red_monitor_program", start_stop=start_stop)
    def remote_monitor(start_stop:DateTime2):
        c.schedule_program("green_boomerang_program", start_stop=start_stop)
        c.schedule_program("red_boomerang_program", start_stop=start_stop)
    def event_callback(start_stop:DateTime2):
        c.defer_action("green_rising_falling", "green_boomerang", DateTime(dt=start_stop.dt1))
        c.defer_action("red_rising_falling", "red_boomerang", DateTime(dt=start_stop.dt1))
        c.expire_action("green_rising_falling", "green_boomerang", DateTime(dt=start_stop.dt2))
        c.expire_action("red_rising_falling", "red_boomerang", DateTime(dt=start_stop.dt2))
    def pivot1(start_stop:DateTime2):
        c.schedule_program("pivot_program1", start_stop=start_stop)
    def pivot2(start_stop:DateTime2):
        c.schedule_program("pivot_program2", start_stop=start_stop)
    def pivot3(start_stop:DateTime2):
        c.schedule_program("pivot_program3", start_stop=start_stop)
    def pivot4(start_stop:DateTime2):
        c.schedule_program("pivot_program4", start_stop=start_stop)
    def file_heartbeat(start_stop:DateTime2):
        c.schedule_program("file_heartbeat_program", start_stop=start_stop)
    def old_school(start_stop:DateTime2):
        [ c.schedule_action(*stuff) for stuff in [
            ('randomly_often', 'red_toggle'),
            ('daily_often', 'green_toggle'),
            ('nightly_often', 'green_toggle')
            ]
        ]
    start = Now.dt() + timedelta(seconds=2)
    start_stop = DateTime2(dt1=start, dt2=start + timedelta(seconds=30))
    for something in [event_callback, pivot1]:
        something(start_stop)



In [ ]:
pprint(all_servers)

In [ ]:
info(all_servers)

In [ ]:
describe(pivot_servers)

In [ ]:
inventory(all_servers)

In [ ]:
schedule(pivot_servers)

In [ ]:
unschedule(pivot_servers)

In [ ]:
run(pivot_servers)

In [ ]:
stop(pivot_servers)

In [ ]:
on(pivot_servers)

In [ ]:
off(pivot_servers)

In [ ]:
gpio_clr(pivot_servers)

In [ ]:
rez=util.Rez(flds={"start_stop": DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=25))})
client(pi3).execute_action_with_rez(action_name="schedule_key_tags", rez=rez)

In [ ]:
rez=util.Rez(flds={"start_stop": DateTime2(dt1=Now.dt() + timedelta(seconds=3), dt2= Now.dt() + timedelta(seconds=25))})
client(local).execute_action_with_rez(action_name="schedule_key_tags2", rez=rez)

In [ ]:
client(pi3).execute_action(action_name="schedule_key_tags3")